# Imports

In [ ]:
import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.optimizers.schedules import ExponentialDecay

from sklearn.metrics import mean_absolute_error as mae
from sklearn.preprocessing import RobustScaler, normalize, MinMaxScaler
from sklearn.model_selection import train_test_split, GroupKFold, KFold

from IPython.display import display

In [ ]:
DEBUG = False
DATA_DIR = '../input/ventilator-pressure-prediction/'

train = pd.read_csv(DATA_DIR+'train.csv')
test = pd.read_csv(DATA_DIR+'test.csv')
submission = pd.read_csv(DATA_DIR+'sample_submission.csv')

if DEBUG:
    train = train[:80*10000]

In [ ]:
def add_features(df):
    df['area'] = df['time_step'] * df['u_in']
    df['area'] = df.groupby('breath_id')['area'].cumsum()
    
    df['u_in_cumsum'] = (df['u_in']).groupby(df['breath_id']).cumsum()
    
    df['u_in_lag'] = df['u_in'].shift(2).fillna(0)
    df['u_in_neg_lag'] = df['u_in'].shift(-2).fillna(0)
    
    df['R'] = df['R'].astype(str)
    df['C'] = df['C'].astype(str)
    df = pd.get_dummies(df)
    return df

train = add_features(train)
test = add_features(test)

In [ ]:
targets = train[['pressure']].to_numpy().reshape(-1, 80)
train.drop(['pressure', 'id', 'breath_id'], axis=1, inplace=True)
test = test.drop(['id', 'breath_id'], axis=1)

# Preprocessing

In [ ]:
sc = RobustScaler()
# sc = MinMaxScaler()
train = sc.fit_transform(train)
test = sc.transform(test)

In [ ]:
train = train.reshape(-1, 80, train.shape[-1])
test = test.reshape(-1, 80, train.shape[-1])

In [ ]:
def get_model():
    model = keras.models.Sequential([
        keras.layers.Input(shape=train.shape[-2:]),
        keras.layers.Bidirectional(keras.layers.LSTM(300, return_sequences=True)),
        keras.layers.Bidirectional(keras.layers.LSTM(250, return_sequences=True)),
        keras.layers.Bidirectional(keras.layers.LSTM(150, return_sequences=True)),
        keras.layers.Bidirectional(keras.layers.LSTM(100, return_sequences=True)),
        keras.layers.Dense(50, activation='selu'),
        keras.layers.Dense(1),
    ])
    model.compile(optimizer="adam", loss="mae", steps_per_execution=32)
    
    return model

In [ ]:
# detect and init the TPU
tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect()
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

with tpu_strategy.scope():
    model = get_model()

In [ ]:
EPOCH = 500
BATCH_SIZE = 1024

scheduler = ExponentialDecay(1e-3, 400*((len(train)*0.8)/BATCH_SIZE), 1e-5)
lr = LearningRateScheduler(scheduler, verbose=0)
es = EarlyStopping(monitor="val_loss", patience=15, verbose=1, mode="min", restore_best_weights=True)

kf = KFold(n_splits=10, shuffle=True, random_state=2021)
test_preds = []
fold_mae = []
for fold, (train_idx, test_idx) in enumerate(kf.split(train, targets)):
    print('-'*50, '>', f'Fold {fold+1}', '<', '-'*50)
    X_train, X_valid = train[train_idx], train[test_idx]
    y_train, y_valid = targets[train_idx], targets[test_idx]
    
#     sc = MinMaxScaler()
#     X_train = sc.fit_transform(X_train)
#     X_valid = sc.transform(X_valid)
#     test_fold = sc.transform(test)
    
    model.fit(
        X_train, y_train,
        validation_data=(X_valid, y_valid),
        epochs=EPOCH,
        batch_size=BATCH_SIZE,
        callbacks=[lr, es],
        verbose=0
    )
    
    y_pred = model.predict(X_valid)
    loss = mae(np.ravel(y_valid), np.ravel(y_pred))
    fold_mae.append(loss)
    print(f"Fold {fold+1} MAE: {loss}")
    #model.save(f'Fold{fold+1} RNN Weights')
    test_preds.append(model.predict(test).squeeze().reshape(-1, 1).squeeze())

In [ ]:
import matplotlib.pyplot as plt
plt.plot(fold_mae)
plt.title("Fold MAE")
plt.xlabel("Fold")
plt.ylabel("MAE")
print(f"Mean MAE: {np.mean(fold_mae)}")

In [ ]:
submission["pressure"] = sum(test_preds)/10
submission.to_csv('submission.csv', index=False)